In [5]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [6]:
engine = create_engine('sqlite:///inventory.db')

In [7]:
for file in os.listdir('data'):
    print(file)

purchases.csv
purchase_prices.csv
vendor_invoice.csv
begin_inventory.csv
.ipynb_checkpoints
end_inventory.csv
sales.csv


In [8]:
# Iterate through files in 'data' directory
for file in os.listdir('data'):
    if file.endswith('.csv'):
        file_path = os.path.join('data', file)
        df = pd.read_csv(file_path)
        print(f"{file}: {df.shape}")

purchases.csv: (2372474, 16)
purchase_prices.csv: (12261, 9)
vendor_invoice.csv: (5543, 10)
begin_inventory.csv: (206529, 9)
end_inventory.csv: (224489, 9)
sales.csv: (12825363, 14)


In [9]:
for file in os.listdir('data'):
    if file.endswith('.csv'):
        file_path = os.path.join('data', file)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"Processing {file} ({size_mb:.2f} MB)...")

Processing purchases.csv (344.83 MB)...
Processing purchase_prices.csv (1.00 MB)...
Processing vendor_invoice.csv (0.49 MB)...
Processing begin_inventory.csv (16.64 MB)...
Processing end_inventory.csv (18.10 MB)...
Processing sales.csv (1522.76 MB)...


In [10]:
def ingest_csv_in_chunks(file_path, table_name, engine, chunk_size=100000):
    """
    Ingests a large CSV file into a database in chunks.

    Parameters:
        file_path (str): Path to the CSV file.
        table_name (str): Name of the target table.
        engine (SQLAlchemy Engine): SQLAlchemy database engine.
        chunk_size (int): Number of rows per chunk.

    Returns:
        None
    """
    first_chunk = True
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        if_exists_mode = 'replace' if first_chunk else 'append'
        chunk.to_sql(table_name, con=engine, if_exists=if_exists_mode, index=False)
        first_chunk = False


In [11]:
for file in os.listdir('data'):
    if file.endswith('.csv'):
        file_path = os.path.join('data', file)
        table_name = os.path.splitext(file)[0]
        print(f"Chunked ingesting: {file}")
        ingest_csv_in_chunks(file_path, table_name, engine)

Chunked ingesting: purchases.csv
Chunked ingesting: purchase_prices.csv
Chunked ingesting: vendor_invoice.csv
Chunked ingesting: begin_inventory.csv
Chunked ingesting: end_inventory.csv
Chunked ingesting: sales.csv


In [12]:
import logging

# Ensure the logs directory exists
os.makedirs("logs", exist_ok=True)

# Configure logging
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,  # DEBUG captures everything
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"  # Append mode; use "w" to overwrite each time
)

In [13]:
import time

def load_raw_data():
    '''Load CSVs from data/ and ingest into DB'''
    data_dir = 'data'
    
    if not os.path.exists(data_dir):
        logging.error(f"Data directory '{data_dir}' does not exist.")
        return
    
    logging.info("Starting to load raw CSV files from 'data' directory.")

    start = time.time()
    
    for file in os.listdir(data_dir):
        if file.endswith('.csv'):
            file_path = os.path.join(data_dir, file)
            table_name = os.path.splitext(file)[0]
            
            logging.info(f"Starting chunked ingestion for file: {file}")
            
            try:
                ingest_csv_in_chunks(file_path, table_name, engine)
                logging.info(f"Successfully ingested file: {file} into table: {table_name}")
            except Exception as e:
                logging.error(f"Failed to ingest file: {file} — Error: {e}")
    
    end = time.time()
    total_time = (end - start) / 60  # in minutes
    logging.info("Finished loading all raw CSV files.")
    logging.info(f"Total Time Taken: {total_time:.2f} minutes")

if __name__ == '__main__':
    load_raw_data()